Get the token from [here](https://api.tumblr.com/console)

In [5]:
from dotenv import dotenv_values
config = dotenv_values(".env")

In [ ]:
import pytumblr

client = pytumblr.TumblrRestClient(
    config['consumer_key'],
    config['consumer_secret'],
    config['oauth_token'],
    config['oauth_secret'],
)

client.info() # Grabs the current user information

In [19]:
blogName = "dailyhistoryposts"
posts = client.posts(blogName,limit=1398) # max 50ish at a time. Need to use offset...
noPosts = posts["total_posts"]

In [21]:
# Save the posts
import json
with open("Logs/posts.json", "w") as f:
    json.dump(posts, f, indent=4)

In [36]:
import re
def getFacts(posts):
    headPattern = re.compile(r"(<h1>[^<]*</h1>)")
    tagsPattern = re.compile(r"(\<[^\>]*\>)")
    hashtagPattern = re.compile(r"^[a-z:]+:")
    datePattern = re.compile(r"[A-Za-z]+\s\d{1,2}")
    facts = {}
    count = 0
    for post in posts["posts"]:
        count += 1
        if post["summary"] == "On This Day In History":
            textRaw = post["body"]
            textWoHeading = re.sub(headPattern, "", textRaw)
            textWoTags = re.sub(tagsPattern, "", textWoHeading)
            text = re.sub(hashtagPattern, "", textWoTags)
            dateMatch = re.findall(datePattern, text)
            if dateMatch:
                date = dateMatch[0]
                existingFacts = facts.get(date, [])
                if not text in existingFacts:
                    existingFacts.append(text)
                    facts[date] = existingFacts
            else:
                pass # On 29/June he forgot to add 'June' to the post, so it falls over. 
    return facts
facts = getFacts(posts)
facts
    

{'July 14': ['July 14th, 1995: The Torres Straight Islander Flag is flown for the first time.'],
 'July 13': ['July 13th, 1977: The Ogaden War, an eight-month long conflict between Somalia and Ethiopia, begins with a Somali invasion.'],
 'July 12': ['July 12th, 1971: The Australian Aboriginal Flag is flown for the first time.ALT'],
 'July 11': ['July 11th, 1893: Mikimoto Kōkichi\xa0obtains the first cultured pearl.'],
 'July 10': ['July 10th, 1806: For the first time, Indian sepoys mutiny against the British East India Company in the Vellore Mutiny.'],
 'July 9': ['July 9th, 1763: Catherine the Great becomes Empress of Russia after the overthrow of her husband/second cousin, Peter III.'],
 'July 8': ['July 8th, 1968: The Chrysler wildcat strike begins in Detroit, Michigan, United States. Part of the Dodge Revolutionary Union Movement (DRUM), a coalition of African-American workers, seeking concessions from Chrysler management and from the United Auto Workers.A wildcat strike is a strik

In [44]:
import os
getFilename = lambda x: f"Logs/PostPage_{x}.json"
offset = 0
batchSize = 50 # max 50ish at a time. Need to use offset...
allPostsReceived = False

blogName = "dailyhistoryposts"
facts = {}

while not allPostsReceived:
    filename = getFilename(offset)
    if os.path.exists(filename):
        with open(filename, "r") as f:
            posts = json.load(f)
    else:
        posts = client.posts(blogName,limit=batchSize, offset=offset)
        with open(f"Logs/PostPage_{offset}.json", "w") as f:
            json.dump(posts, f, indent=4)
    # Extract the facts from the posts
    newFacts = getFacts(posts)
    # Append the new facts to the existing facts, and then save the facts back
    for date in list(newFacts.keys()):
        existingFacts = facts.get(date, [])
        for fact in newFacts[date]:
            if not fact in existingFacts:
                existingFacts.append(fact)
        facts[date] = existingFacts
    offset += batchSize
    noPosts = posts["total_posts"]
    if offset >= noPosts:
        allPostsReceived = True

In [42]:
with open("facts.json", "w") as f:
    json.dump(facts, f, indent=4)

In [40]:
facts['January 5']

['January 5th, 1953: Waiting for Godot, considered the most significant English-language play of the 20th century, is premiered in Paris.',
 'January 5th, 1914: The Ford Motor Company announces an eight-hour workday and minimum daily wage of $5 in salary plus bonuses.']